# On dynamic range: 'roundoff errors' and the FFT

### Background
Accuracy of the FFT has been a consideration since (at least) the initial implementation of the Cooley-Tukey algorithm, there are a handful papers on the subject going right back to the 60's - 'accuracy' and 'round-off error' being the most relevant search terms. The best recent references I could find are:

* [Accuracy of the Discrete Fourier Transform and the Fast Fourier Transform (Schatzman 1996)](https://doi.org/10.1137/S1064827593247023) - a relatively recent review of the theory

* [Implementing FFTs in Practice (Johnson et al. 2008)](http://cnx.org/content/m16336/) - from the authors of FFTW

The consensus seems to be that the relative error from a (carefully implemented and tuned) FFT of input length N grows as $O(\log{N})$ in the worst case, and $O(\sqrt{\log{N}})$ on average for the more amenable case of random input data. This applies to one-dimensional FFT's, but (perhaps surprisingly) the error-growth curve for a 2-dimensional FFT is likely of the same order and with a scaling constant only marginally larger (cf $\S6$, Schatzman 1996).


However, it's worth noting that the typical results (as reported in the [FFTW accuracy benchmarks](http://www.fftw.org/accuracy/comments.html)) are obtained by testing with random data (computed with arbitrarily high precision and then compare with results computed via standard single / double floating precision). So when considering a radio-field with a bright source and a faint source, we may be in the 'worst-case' regime. On the other hand, if we are recovering a weak signal from many noisy visibilities then we very close to emulating the 'typical' random-noise regime.

This is a typical benchmark plot for FFTW:
![Illustrative FFTW benchmark plot](http://www.fftw.org/accuracy/Ryzen-7-3.6GHz/ryzen.1d.scxx.acc.p2.png)

This has a similar shape to the best-case curves from Schatzman 1996 (cf $\S3.4$).
For an FFT of length $N$ we expect a relative error in the random-noise 'typical' cose of order $$\sqrt{\log{N}}\epsilon = 3\epsilon$$
where $\epsilon$ is the machine accuracy --- recall this is  $\frac{1}{2^{24}}$ in the single-precision case.
So we can run a quick calculation, generate our own plot, and check if the FFTW plot agrees with our understanding of the theory:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
import numpy as np
from matplotlib.ticker import LogLocator


N = np.asarray([float(2**i) for i in range(4,18)])
epsilon = 1./ 2**26
arbitrary_scaling_factor = 0.6 # cf Schatzman 1996, error function 3b

def fft_err_factor_for_random_inputs(N):
    return arbitrary_scaling_factor* np.sqrt(np.log2(N))

def fft_err_factor_worst_case(N):
    return  np.log2(N)#  worst case

plt.plot(N, 
         fft_err_factor_for_random_inputs(N), 
#          fft_err_factor_for_random_inputs(N)*epsilon,
         label='Typical case (random inputs)')

plt.plot(N, fft_err_factor_worst_case(N), label='Worst case')

ax = plt.gca()

ax.set_xscale('log', basex=2)
# ax.set_yscale('log')

plt.xlim(min(N), max(N))
# plt.axhline(1e-7, ls=':')
plt.xlabel('N')
plt.ylabel('Relative error')
plt.axvline(1024, ls=':', label='N=1024')

plt.legend(loc='best')

Result - not a perfect agreement, but the 'typical case' plot is in the right ballpark (hard to judge exactly without a better idea of the scaling on the FFT benchmark plot).

So if things are good, relative precision due to the FFT is only a factor of ~2 worse than in any other mathematical operation - we lose about 1 bit of precision - and that's pretty flat.

If we hit the worst case, then we lose about a factor of 10 for N=2^10=1024 - or about 3-4 bits of precision depending on the size of our transform, for a typical transform size of $2^8$ -  $2^{16}$ (256 - 65536).

## Test-case 1: Bright and faint sources, no noise.

So, let's see if things behave as expected in the 1-d case. We'll start with a bright source and faint source (modelled by single non-zero pixels), then perform the FFT to get our 1-d 'visibilities', then drop to single-precision and see if the faint source gets lost to roundoff error.

Note that numpy only implements double precision FFTs, so we switch to the SciPy [implementation](https://docs.scipy.org/doc/scipy/reference/fftpack.html) which also has a single-precision option.

In [ ]:
import numpy as np
import scipy.fftpack as fftpack

In [ ]:
N = 2**12
bright_src_flux = 1.
faint_src_flux = bright_src_flux / 2**30
input_data = np.zeros(N, dtype=np.complex128) 
bright_src_idx  = int(N/2 + N/3 + 7)
faint_src_idx = int(N/2 + N/4 +5)

bright_src_region = slice(bright_src_idx-10, bright_src_idx+10)
faint_src_region = slice(faint_src_idx-10, faint_src_idx+10)

input_data[bright_src_idx] = bright_src_flux
input_data[faint_src_idx] = faint_src_flux

input_idx = np.arange(N)
for idx in input_data.nonzero()[0]:
    print(idx, input_data[idx])
input_nz_idx = input_data.nonzero()

# plt.scatter(input_idx[input_nz_idx], input_data[input_nz_idx])
plt.plot(input_data[faint_src_region])
plt.xlabel('Pixel index')
plt.ylabel('Component value')

In [ ]:
vis = fftpack.fft(input_data)
seed = 42
rstate = np.random.RandomState(seed)
noise = rstate.normal(loc=0, scale=faint_src_flux*(np.sqrt(N)/(8.)), size=(len(vis), 2))
noisy_vis = vis + (noise[:, 0] + 1j * noise[:, 1])

vis_f = np.asarray(vis, dtype=np.complex64)
noisy_vis_f = np.asarray(noisy_vis, dtype=np.complex64)

In [ ]:
# noise

In [ ]:
output = fftpack.ifft(vis)
output_f = fftpack.ifft(vis_f)
noisy_output = fftpack.ifft(noisy_vis)
noisy_output_f = fftpack.ifft(noisy_vis_f)
print(output.dtype)
# for idx in output.nonzero()[0]:
#     print(idx, output[idx])

In [ ]:
ax = plt.gca()
ax.plot(input_idx[faint_src_region],np.real(output[faint_src_region]), label='Double prec.')
ax.plot(input_idx[faint_src_region],np.real(output_f[faint_src_region]), label='Single prec.')

# plt.yscale('log')
ax.set_xlabel('Pixel index')
ax.set_ylabel('Component value')
ax.axvline(faint_src_idx, ls=':')
ax.axhline(faint_src_flux, ls=':')
# ax.set_ylim( -faint_src_flux, 2*faint_src_flux)
ax.set_title('Faint source region')
ax.legend(loc='best')

In [ ]:
ax = plt.gca()
ax.scatter(input_idx[bright_src_region],np.real(output[bright_src_region]))
ax.axvline(bright_src_idx, ls=':')